In [2]:
import pandas as pd
import numpy as np

from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players

pd.set_option('display.max_columns', None)

In [3]:
OVERVIEWS = {
    # Worlds
    'Season 1 World Championship': 'Season 1 World Championship',
    'Season 2 World Championship': 'Season 2 World Championship',
    'Season 3 World Championship': 'Season 3 World Championship',
    '2014 Season World Championship': '2014 Season World Championship',
    '2015 Season World Championship': '2015 Season World Championship',
    '2016 Season World Championship': '2016 Season World Championship',
    '2017 Season World Championship/Play-In': '2017 Season World Championship',
    '2017 Season World Championship/Main Event': '2017 Season World Championship',
    '2018 Season World Championship/Play-In': '2018 Season World Championship',
    '2018 Season World Championship/Main Event': '2018 Season World Championship',
    '2019 Season World Championship/Play-In': '2019 Season World Championship',
    '2019 Season World Championship/Main Event': '2019 Season World Championship',
    '2020 Season World Championship/Play-In': '2020 Season World Championship',
    '2020 Season World Championship/Main Event': '2020 Season World Championship',
    '2021 Season World Championship/Play-In': '2021 Season World Championship',
    '2021 Season World Championship/Main Event': '2021 Season World Championship',
    # MSI
    '2015 Mid-Season Invitational': '2015 Mid-Season Invitational',
    '2016 Mid-Season Invitational': '2016 Mid-Season Invitational',
    '2017 Mid-Season Invitational/Play-In': '2017 Mid-Season Invitational',
    '2017 Mid-Season Invitational/Main Event': '2017 Mid-Season Invitational',
    '2018 Mid-Season Invitational/Play-In': '2018 Mid-Season Invitational',
    '2018 Mid-Season Invitational/Main Event': '2018 Mid-Season Invitational',
    '2019 Mid-Season Invitational/Play-In': '2019 Mid-Season Invitational',
    '2019 Mid-Season Invitational/Main Event': '2019 Mid-Season Invitational',
    '2021 Mid-Season Invitational': '2021 Mid-Season Invitational',
    '2022 Mid-Season Invitational': '2022 Mid-Season Invitational',
    # LCK
    'Champions/2012 Season/Spring': 'Champions/2012 Season/Spring',
    'Champions/2012 Season/Summer': 'Champions/2012 Season/Summer',
    'Champions/2013 Season/Winter': 'Champions/2013 Season/Winter',
    'Champions/2013 Season/Spring': 'Champions/2013 Season/Spring',
    'Champions/2013 Season/Summer': 'Champions/2013 Season/Summer',
    'Champions/2014 Season/Winter Season': 'Champions/2014 Season/Winter',
    'Champions/2014 Season/Spring Season': 'Champions/2014 Season/Spring',
    'Champions/2014 Season/Summer Season': 'Champions/2014 Season/Summer',
    'Champions/2015 Season/Spring Season': 'Champions/2015 Season/Spring',
    'Champions/2015 Season/Spring Playoffs': 'Champions/2015 Season/Spring',
    'Champions/2015 Season/Summer Season': 'Champions/2015 Season/Summer',
    'Champions/2015 Season/Summer Playoffs': 'Champions/2015 Season/Summer',
    'LCK/2016 Season/Spring Season': 'LCK/2016 Season/Spring',
    'LCK/2016 Season/Spring Playoffs': 'LCK/2016 Season/Spring',
    'LCK/2016 Season/Summer Season': 'LCK/2016 Season/Summer',
    'LCK/2016 Season/Summer Playoffs': 'LCK/2016 Season/Summer',
    'LCK/2017 Season/Spring Season': 'LCK/2017 Season/Spring',
    'LCK/2017 Season/Spring Playoffs': 'LCK/2017 Season/Spring',
    'LCK/2017 Season/Summer Season': 'LCK/2017 Season/Summer',
    'LCK/2017 Season/Summer Playoffs': 'LCK/2017 Season/Summer',
    'LCK/2018 Season/Spring Season': 'LCK/2018 Season/Spring',
    'LCK/2018 Season/Spring Playoffs': 'LCK/2018 Season/Spring',
    'LCK/2018 Season/Summer Season': 'LCK/2018 Season/Summer',
    'LCK/2018 Season/Summer Playoffs': 'LCK/2018 Season/Summer',
    'LCK/2019 Season/Spring Season': 'LCK/2019 Season/Spring',
    'LCK/2019 Season/Spring Playoffs': 'LCK/2019 Season/Spring',
    'LCK/2019 Season/Summer Season': 'LCK/2019 Season/Summer',
    'LCK/2019 Season/Summer Playoffs': 'LCK/2019 Season/Summer',
    'LCK/2020 Season/Spring Season': 'LCK/2020 Season/Spring',
    'LCK/2020 Season/Spring Playoffs': 'LCK/2020 Season/Spring',
    'LCK/2020 Season/Summer Season': 'LCK/2020 Season/Summer',
    'LCK/2020 Season/Summer Playoffs': 'LCK/2020 Season/Summer',
    'LCK/2021 Season/Spring Season': 'LCK/2021 Season/Spring',
    'LCK/2021 Season/Spring Playoffs': 'LCK/2021 Season/Spring',
    'LCK/2021 Season/Summer Season': 'LCK/2021 Season/Summer',
    'LCK/2021 Season/Summer Playoffs': 'LCK/2021 Season/Summer',
    'LCK/2022 Season/Spring Season': 'LCK/2022 Season/Spring',
    'LCK/2022 Season/Spring Playoffs': 'LCK/2022 Season/Spring',
    'LCK/2022 Season/Summer Season': 'LCK/2022 Season/Summer',
    'LCK/2022 Season/Summer Playoffs': 'LCK/2022 Season/Summer',
    # LPL
    'LPL/2013 Season/Spring Season': 'LPL/2013 Season/Spring',
    'LPL/2013 Season/Spring Playoffs': 'LPL/2013 Season/Spring',
    'LPL/2013 Season/Summer Season': 'LPL/2013 Season/Summer',
    'LPL/2013 Season/Summer Playoffs': 'LPL/2013 Season/Summer',
    'LPL/2014 Season/Spring Season': 'LPL/2014 Season/Spring',
    'LPL/2014 Season/Spring Playoffs': 'LPL/2014 Season/Spring',
    'LPL/2014 Season/Summer Season': 'LPL/2014 Season/Summer',
    'LPL/2014 Season/Summer Playoffs': 'LPL/2014 Season/Summer',
    'LPL/2015 Season/Spring Season': 'LPL/2015 Season/Spring',
    'LPL/2015 Season/Spring Playoffs': 'LPL/2015 Season/Spring',
    'LPL/2015 Season/Summer Season': 'LPL/2015 Season/Summer',
    'LPL/2015 Season/Summer Playoffs': 'LPL/2015 Season/Summer',
    'LPL/2016 Season/Spring Season': 'LPL/2016 Season/Spring',
    'LPL/2016 Season/Spring Playoffs': 'LPL/2016 Season/Spring',
    'LPL/2016 Season/Summer Season': 'LPL/2016 Season/Summer',
    'LPL/2016 Season/Summer Playoffs': 'LPL/2016 Season/Summer',
    'LPL/2017 Season/Spring Season': 'LPL/2017 Season/Spring',
    'LPL/2017 Season/Spring Playoffs': 'LPL/2017 Season/Spring',
    'LPL/2017 Season/Summer Season': 'LPL/2017 Season/Summer',
    'LPL/2017 Season/Summer Playoffs': 'LPL/2017 Season/Summer',
    'LPL/2018 Season/Spring Season': 'LPL/2018 Season/Spring',
    'LPL/2018 Season/Spring Playoffs': 'LPL/2018 Season/Spring',
    'LPL/2018 Season/Summer Season': 'LPL/2018 Season/Summer',
    'LPL/2018 Season/Summer Playoffs': 'LPL/2018 Season/Summer',
    'LPL/2019 Season/Spring Season': 'LPL/2019 Season/Spring',
    'LPL/2019 Season/Spring Playoffs': 'LPL/2019 Season/Spring',
    'LPL/2019 Season/Summer Season': 'LPL/2019 Season/Summer',
    'LPL/2019 Season/Summer Playoffs': 'LPL/2019 Season/Summer',
    'LPL/2020 Season/Spring Season': 'LPL/2020 Season/Spring',
    'LPL/2020 Season/Spring Playoffs': 'LPL/2020 Season/Spring',
    'LPL/2020 Season/Summer Season': 'LPL/2020 Season/Summer',
    'LPL/2020 Season/Summer Playoffs': 'LPL/2020 Season/Summer',
    'LPL/2021 Season/Spring Season': 'LPL/2021 Season/Spring',
    'LPL/2021 Season/Spring Playoffs': 'LPL/2021 Season/Spring',
    'LPL/2021 Season/Summer Season': 'LPL/2021 Season/Summer',
    'LPL/2021 Season/Summer Playoffs': 'LPL/2021 Season/Summer',
    'LPL/2022 Season/Spring Season': 'LPL/2022 Season/Spring',
    'LPL/2022 Season/Spring Playoffs': 'LPL/2022 Season/Spring',
    'LPL/2022 Season/Summer Season': 'LPL/2022 Season/Summer',
    'LPL/2022 Season/Summer Playoffs': 'LPL/2022 Season/Summer',
    # LEC
    'EU LCS/Season 3/Spring Season': 'EU LCS/Season 3/Spring',
    'EU LCS/Season 3/Spring Playoffs': 'EU LCS/Season 3/Spring',
    'EU LCS/Season 3/Summer Season': 'EU LCS/Season 3/Summer',
    'EU LCS/Season 3/Summer Playoffs': 'EU LCS/Season 3/Summer',
    'EU LCS/2014 Season/Spring Season': 'EU LCS/2014 Season/Spring',
    'EU LCS/2014 Season/Spring Playoffs': 'EU LCS/2014 Season/Spring',
    'EU LCS/2014 Season/Summer Season': 'EU LCS/2014 Season/Summer',
    'EU LCS/2014 Season/Summer Playoffs': 'EU LCS/2014 Season/Summer',
    'EU LCS/2015 Season/Spring Season': 'EU LCS/2015 Season/Spring',
    'EU LCS/2015 Season/Spring Playoffs': 'EU LCS/2015 Season/Spring',
    'EU LCS/2015 Season/Summer Season': 'EU LCS/2015 Season/Summer',
    'EU LCS/2015 Season/Summer Playoffs': 'EU LCS/2015 Season/Summer',
    'EU LCS/2016 Season/Spring Season': 'EU LCS/2016 Season/Spring',
    'EU LCS/2016 Season/Spring Playoffs': 'EU LCS/2016 Season/Spring',
    'EU LCS/2016 Season/Summer Season': 'EU LCS/2016 Season/Summer',
    'EU LCS/2016 Season/Summer Playoffs': 'EU LCS/2016 Season/Summer',
    'EU LCS/2017 Season/Spring Season': 'EU LCS/2017 Season/Spring',
    'EU LCS/2017 Season/Spring Playoffs': 'EU LCS/2017 Season/Spring',
    'EU LCS/2017 Season/Summer Season': 'EU LCS/2017 Season/Summer',
    'EU LCS/2017 Season/Summer Playoffs': 'EU LCS/2017 Season/Summer',
    'EU LCS/2018 Season/Spring Season': 'EU LCS/2018 Season/Spring',
    'EU LCS/2018 Season/Spring Playoffs': 'EU LCS/2018 Season/Spring',
    'EU LCS/2018 Season/Summer Season': 'EU LCS/2018 Season/Summer',
    'EU LCS/2018 Season/Summer Playoffs': 'EU LCS/2018 Season/Summer',
    'LEC/2019 Season/Spring Season': 'LEC/2019 Season/Spring',
    'LEC/2019 Season/Spring Playoffs': 'LEC/2019 Season/Spring',
    'LEC/2019 Season/Summer Season': 'LEC/2019 Season/Summer',
    'LEC/2019 Season/Summer Playoffs': 'LEC/2019 Season/Summer',
    'LEC/2020 Season/Spring Season': 'LEC/2020 Season/Spring',
    'LEC/2020 Season/Spring Playoffs': 'LEC/2020 Season/Spring',
    'LEC/2020 Season/Summer Season': 'LEC/2020 Season/Summer',
    'LEC/2020 Season/Summer Playoffs': 'LEC/2020 Season/Summer',
    'LEC/2021 Season/Spring Season': 'LEC/2021 Season/Spring',
    'LEC/2021 Season/Spring Playoffs': 'LEC/2021 Season/Spring',
    'LEC/2021 Season/Summer Season': 'LEC/2021 Season/Summer',
    'LEC/2021 Season/Summer Playoffs': 'LEC/2021 Season/Summer',
    'LEC/2022 Season/Spring Season': 'LEC/2022 Season/Spring',
    'LEC/2022 Season/Spring Playoffs': 'LEC/2022 Season/Spring',
    'LEC/2022 Season/Summer Season': 'LEC/2022 Season/Summer',
    'LEC/2022 Season/Summer Playoffs': 'LEC/2022 Season/Summer',
    # LCS
    'NA LCS/Season 3/Spring Season': 'NA LCS/Season 3/Spring',
    'NA LCS/Season 3/Spring Playoffs': 'NA LCS/Season 3/Spring',
    'NA LCS/Season 3/Summer Season': 'NA LCS/Season 3/Summer',
    'NA LCS/Season 3/Summer Playoffs': 'NA LCS/Season 3/Summer',
    'NA LCS/2014 Season/Spring Season': 'NA LCS/2014 Season/Spring',
    'NA LCS/2014 Season/Spring Playoffs': 'NA LCS/2014 Season/Spring',
    'NA LCS/2014 Season/Summer Season': 'NA LCS/2014 Season/Summer',
    'NA LCS/2014 Season/Summer Playoffs': 'NA LCS/2014 Season/Summer',
    'NA LCS/2015 Season/Spring Season': 'NA LCS/2015 Season/Spring',
    'NA LCS/2015 Season/Spring Playoffs': 'NA LCS/2015 Season/Spring',
    'NA LCS/2015 Season/Summer Season': 'NA LCS/2015 Season/Summer',
    'NA LCS/2015 Season/Summer Playoffs': 'NA LCS/2015 Season/Summer',
    'NA LCS/2016 Season/Spring Season': 'NA LCS/2016 Season/Spring',
    'NA LCS/2016 Season/Spring Playoffs': 'NA LCS/2016 Season/Spring',
    'NA LCS/2016 Season/Summer Season': 'NA LCS/2016 Season/Summer',
    'NA LCS/2016 Season/Summer Playoffs': 'NA LCS/2016 Season/Summer',
    'NA LCS/2017 Season/Spring Season': 'NA LCS/2017 Season/Spring',
    'NA LCS/2017 Season/Spring Playoffs': 'NA LCS/2017 Season/Spring',
    'NA LCS/2017 Season/Summer Season': 'NA LCS/2017 Season/Summer',
    'NA LCS/2017 Season/Summer Playoffs': 'NA LCS/2017 Season/Summer',
    'NA LCS/2018 Season/Spring Season': 'NA LCS/2018 Season/Spring',
    'NA LCS/2018 Season/Spring Playoffs': 'NA LCS/2018 Season/Spring',
    'NA LCS/2018 Season/Summer Season': 'NA LCS/2018 Season/Summer',
    'NA LCS/2018 Season/Summer Playoffs': 'NA LCS/2018 Season/Summer',
    'LCS/2019 Season/Spring Season': 'LCS/2019 Season/Spring',
    'LCS/2019 Season/Spring Playoffs': 'LCS/2019 Season/Spring',
    'LCS/2019 Season/Summer Season': 'LCS/2019 Season/Summer',
    'LCS/2019 Season/Summer Playoffs': 'LCS/2019 Season/Summer',
    'LCS/2020 Season/Spring Season': 'LCS/2020 Season/Spring',
    'LCS/2020 Season/Spring Playoffs': 'LCS/2020 Season/Spring',
    'LCS/2020 Season/Summer Season': 'LCS/2020 Season/Summer',
    'LCS/2020 Season/Summer Playoffs': 'LCS/2020 Season/Summer',
    'LCS/2021 Season/Spring Season': 'LCS/2021 Season/Spring',
    'LCS/2021 Season/Mid-Season Showdown': 'LCS/2021 Season/Spring',
    'LCS/2021 Season/Summer Season': 'LCS/2021 Season/Summer',
    'LCS/2021 Season/Championship': 'LCS/2021 Season/Summer',
    'LCS/2022 Season/Spring Season': 'LCS/2022 Season/Spring',
    'LCS/2022 Season/Spring Playoffs': 'LCS/2022 Season/Spring',
    'LCS/2022 Season/Summer Season': 'LCS/2022 Season/Summer',
    'LCS/2022 Season/Championship': 'LCS/2022 Season/Summer',
}

In [4]:
def change_data(games):
    df = pd.DataFrame()
    columns = [
        'OverviewPage', 'Team', 'Opponent', 'Win', 'Loss', 'DateTime UTC',
        'Gamelength', 'Gamelength Number', 'Gold', 'Kills'
    ]
    for i, row in enumerate(games.itertuples()):
        data = {
            'OverviewPage': [row.OverviewPage] * 2,
            'Team': [row.Team1, row.Team2],
            'Opponent': [row.Team2, row.Team1],
            'Win': [],
            'Loss': [],
            'DateTime UTC': [row._6] * 2,
            'Gamelength': [row.Gamelength] * 2,
            'Gamelength Number': [row._11] * 2,
            'TeamBans': [row.Team1Bans, row.Team2Bans],
            'OpponentBans': [row.Team2Bans, row.Team1Bans],
            'TeamPicks': [row.Team1Picks, row.Team2Picks],
            'OpponentPicks': [row.Team2Picks, row.Team1Picks],
            'TeamPlayers': [row.Team1Players, row.Team2Players],
            'OpponentPlayers': [row.Team2Players, row.Team1Players],
            'TeamDragons': [row.Team1Dragons, row.Team2Dragons],
            'OpponentDragons': [row.Team2Dragons, row.Team1Dragons],
            'TeamBarons': [row.Team1Barons, row.Team2Barons],
            'OpponentBarons': [row.Team2Barons, row.Team1Barons],
            'TeamTowers': [row.Team1Towers, row.Team2Towers],
            'OpponentTowers': [row.Team2Towers, row.Team1Towers],
            'TeamGold': [row.Team1Gold, row.Team2Gold],
            'OpponentGold': [row.Team2Gold, row.Team1Gold],
            'TeamKills': [row.Team1Kills, row.Team2Kills],
            'OpponentKills': [row.Team2Kills, row.Team1Kills],
            'TeamRiftHeralds': [row.Team1RiftHeralds, row.Team2RiftHeralds],
            'OpponentHeralds': [row.Team2RiftHeralds, row.Team1RiftHeralds],
            'TeamInhibitors': [row.Team1Inhibitors, row.Team2Inhibitors],
            'OpponentInhibitors': [row.Team2Inhibitors, row.Team1Inhibitors],
        }
        win = 1 if row.Team1 == row.WinTeam else 0
        data['Win'].extend([win, 1 - win])
        data['Loss'].extend([1 - win, win])

        df = pd.concat([df, pd.DataFrame(data)])
    return df.reset_index(drop=True)

def get_team_stats(games):
    grouped = games.groupby('Team')
    df = pd.DataFrame()
    df['Win'] = grouped['Win'].sum()
    df['Loss'] = grouped['Loss'].sum()
    df['Gold'] = grouped['Gold'].mean()
    df['Kills'] = grouped['Kills'].mean()
    df['Deaths'] = grouped['Deaths'].mean()
    df['Gamelength Number'] = grouped['Gamelength Number'].mean()
    df['Win Gamelength Number'] = \
        grouped[['Win', 'Gamelength Number']].apply(
            lambda x: (x['Win'] * x['Gamelength Number']).sum()
        ) / df['Win']
    df['Loss Gamelength Number'] = \
        grouped[['Loss', 'Gamelength Number']].apply(
            lambda x: (x['Loss'] * x['Gamelength Number']).sum()
        ) / df['Loss']
    df['Games'] = grouped['Win'].count()
    df['WinRate'] = df['Win'] / df['Games']
    df['KPM'] = grouped['KPM'].mean()
    df['KDPM'] = grouped['KDPM'].mean()
    df['GPM'] = grouped['GPM'].mean()
    df['GDPM'] = grouped['GDPM'].mean()
    df['OverviewPage'] = grouped['OverviewPage'].last()
    df['Winner'] = 0

    columns = [
        'OverviewPage', 'Games', 'Win', 'Loss', 'WinRate', 'Gamelength Number',
        'Win Gamelength Number', 'Loss Gamelength Number', 'Gold', 'Kills',
        'GPM', 'GDPM', 'KPM', 'KDPM', 'Winner'
    ]

    return df[columns]

In [5]:
LEAGUES = [
    'WCS', 'MSI', 'LTC', 'LCK', 'LPL', 'EU LCS', 'LEC', 'NA LCS', 'LCS'
]

In [66]:
leagues = pd.DataFrame()
for league in LEAGUES:
    temp = get_leagues(where=f'L.League_Short="{league}"')
    leagues = pd.concat([leagues, temp])
leagues = leagues.reset_index(drop=True)
leagues

,League,League Short,Region,Level,IsOfficial
0,World Championship,WCS,International,Primary,Yes
1,Mid-Season Invitational,MSI,International,Primary,Yes
2,LoL The Champions,LTC,Korea,Primary,Yes
3,LoL Champions Korea,LCK,Korea,Primary,Yes
4,Tencent LoL Pro League,LPL,China,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,LoL European Championship,LEC,Europe,Primary,Yes
7,North America League Championship Series,NA LCS,North America,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes


In [78]:
def filter_tournaments(tournaments):
    for row in tournaments.itertuples():
        if row.OverviewPage not in OVERVIEWS.keys():
            tournaments = tournaments.drop(index=row.Index)
    return tournaments.reset_index(drop=True)

tournaments = pd.DataFrame()
for league in leagues['League Short']:
    if league in ['WCS']:
        tournament = get_tournaments(
            where=f'L.League_Short="{league}" and T.Region="International"'
        )
    elif league == 'MSI':
        tournament = get_tournaments(
            where=f'L.League_Short="{league}"'
        )
    else:
        tournament = get_tournaments(
            where=f'L.League_Short="{league}" and T.IsQualifier=0'
        )
    tournaments = pd.concat([tournaments, tournament])
    
tournaments = tournaments.sort_values(
    by=['Year', 'DateStart', 'Date']
).reset_index(drop=True)
tournaments = filter_tournaments(tournaments)
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,Worlds Season 1,Season 1 World Championship,2011-06-18,2011-06-20,World Championship,International,Offline,Worlds Season 1,None,None,Primary,0,0,1,2011,1,1
1,Champions 2012 Spring,Champions/2012 Season/Spring,2012-03-21,2012-05-19,LoL The Champions,Korea,Offline,Champions 2012 Spring,Spring,1,Primary,0,0,1,2012,1,1
2,Champions 2012 Summer,Champions/2012 Season/Summer,2012-07-04,2012-09-08,LoL The Champions,Korea,Offline,Champions 2012 Summer,Summer,2,Primary,0,0,1,2012,1,1
3,Worlds Season 2,Season 2 World Championship,2012-10-04,2012-10-13,World Championship,International,Offline,Worlds Season 2,None,None,Primary,0,0,1,2012,1,1
4,Champions 2013 Winter,Champions/2013 Season/Winter,2012-11-09,2013-02-02,LoL The Champions,Korea,Offline,Champions 2013 Winter,Winter,1,Primary,0,0,1,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,LCS 2022 Summer,LCS/2022 Season/Summer Season,2022-06-17,2022-08-14,League of Legends Championship Series,North America,None,LCS 2022 Summer,Summer,2,Primary,0,0,1,2022,1,1
182,LPL 2022 Summer Playoffs,LPL/2022 Season/Summer Playoffs,2022-08-16,2022-09-01,Tencent LoL Pro League,China,Offline,LPL 2022 Summer Playoffs,Summer,2,Primary,0,1,1,2022,1,1
183,LCK 2022 Summer Playoffs,LCK/2022 Season/Summer Playoffs,2022-08-17,2022-08-28,LoL Champions Korea,Korea,Offline/Online,LCK 2022 Summer Playoffs,Summer,2,Primary,0,1,1,2022,1,1
184,LCS 2022 Championship,LCS/2022 Season/Championship,2022-08-20,2022-09-11,League of Legends Championship Series,North America,Online/Offline,LCS 2022 Championship,Summer,2,Primary,0,1,1,2022,1,1


In [82]:
def change_overviews(games):
    games['OverviewPage'] = games['OverviewPage'].apply(lambda x: OVERVIEWS[x])
    return games

scoreboard_games = pd.DataFrame()
for overview in tournaments['OverviewPage']:
    temp = get_scoreboard_games(where=f'T.OverviewPage="{overview}"')
    if temp is None:
        print(f'{overview} is None')
        break
    scoreboard_games = pd.concat([scoreboard_games, temp])
scoreboard_games = scoreboard_games.reset_index(drop=True)
scoreboard_games = change_overviews(scoreboard_games)
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,Season 1 World Championship,against All authority,Epik Gamer,Epik Gamer,against All authority,2011-06-18 08:30:00,0,1,2,45:09,45.150000,"Akali,Vladimir,Nunu","Rumble,Nidalee,Twisted Fate","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Singed,Warwick,Annie,Vayne,Janna","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Dyrus,Dan Dinh,Salce,Westrice,Doublelift",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 1 World Championship_Group Stage_1_1,Season 1 World Championship_Group Stage_1,None,0
1,Season 1 World Championship,against All authority,Fnatic,against All authority,Fnatic,2011-06-18 11:00:00,1,0,1,41:18,41.300000,"Alistar,Rumble,Twisted Fate","Irelia,Karthus,Nidalee","Gragas,Nunu,Anivia,Ashe,Sona","Vladimir,Amumu,Brand,Miss Fortune,Taric","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","wewillfailer,Cyanide,Shushei,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 1 World Championship_Group Stage_3_1,Season 1 World Championship_Group Stage_3,None,0
2,Season 1 World Championship,against All authority,Fnatic,Fnatic,against All authority,2011-06-20 22:00:00,1,2,2,43:57,43.950000,"Alistar,Gragas,Anivia","Rumble,Nunu,Amumu","Malzahar,Jarvan IV,Irelia,Corki,Janna","Brand,Warwick,Karthus,Ashe,Sona","MoMa,Linak,sOAZ,YellOwStaR,Kujaa","Shushei,Cyanide,xPeke,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 1 World Championship_Bracket Stage_9_2,Season 1 World Championship_Bracket Stage_9,None,0
3,Season 1 World Championship,against All authority,Fnatic,Fnatic,against All authority,2011-06-20 11:30:00,0,1,2,31:46,31.766667,"Alistar,Ashe,Vladimir","Rumble,Nunu,Amumu","Nidalee,Warwick,Anivia,Miss Fortune,Sona","Gragas,Jarvan IV,Malzahar,Corki,Janna","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Shushei,Cyanide,xPeke,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 1 World Championship_Bracket Stage_6_1,Season 1 World Championship_Bracket Stage_6,None,0
4,Season 1 World Championship,against All authority,Team gamed!de,against All authority,Team gamed!de,2011-06-19 09:30:00,2,0,1,32:53,32.883333,"Vayne,Sona,Rumble","Irelia,Karthus,Nidalee","Warwick,Nunu,Malzahar,Ashe,Janna","Brand,Amumu,Anivia,Tristana,Taric","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Reyk,Zylor,Kev1n,CandyPanda,Nyph",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 1 World Championship_Bracket Stage_1_1,Season 1 World Championship_Bracket Stage_1,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16866,LEC/2022 Season/Summer,Rogue (European Team),G2 Esports,G2 Esports,Rogue (European Team),2022-09-03 17:03:00,0,3,2,25:48,25.800000,"Jarvan IV,Senna,Sejuani,Varus,Renata Glasc","Poppy,Lucian,Yuumi,Kalista,Samira","Ornn,Wukong,Azir,Zeri,Braum","Yone,Trundle,Sylas,Draven,Rakan","Odoamne,Malrang,Larssen,Comp,Trymbi","BrokenBlade,Jankos,caPs,Flakked,Targamas",0.0,3.0,0.0,1.0,1.0,8.0,40647.0,53026.0,8.0,17.0,1.0,1.0,0.0,1.0,12.15,LEC/2022 Season/Summer Playoffs_Semifinals_1_3,LEC/2022 Season/Summer Playoffs_Semifinals_1,None,0
16867,LEC/2022 Season/Summer,Rogue (European Team),G2 Esports,G2 Esports,Rogue (European Team),2022-09-03 16:01:00,0,2,2,38:27,38.450000,"Jarvan IV,Draven,Sejuani,Ornn,Ahri","Poppy,Lucian,Kalista,Trundle,Zeri","Renekton,Xin Zhao,Azir,Sivir,Yuumi","Aatrox,Wukong,Varus,Seraphine,Senna","Odoamne,Malrang,Larssen,Comp,Trymbi","BrokenBlade,Jankos,caPs,Flakked,Targamas",1.0,4.0,1.0,0.0,6.0,7.0,65158.0,67141.0,9.0,12.0,1.0,1.0,0.0,1.0,12.15,LEC/2022 Sea

In [86]:
games = change_data(scoreboard_games)
print(scoreboard_games.shape[0] * 2 == games.shape[0])
games

True


,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,TeamBans,OpponentBans,TeamPicks,OpponentPicks,TeamPlayers,OpponentPlayers,TeamDragons,OpponentDragons,TeamBarons,OpponentBarons,TeamTowers,OpponentTowers,TeamGold,OpponentGold,TeamKills,OpponentKills,TeamRiftHeralds,OpponentHeralds,TeamInhibitors,OpponentInhibitors
0,Season 1 World Championship,against All authority,Epik Gamer,0,1,2011-06-18 08:30:00,45:09,45.150000,"Akali,Vladimir,Nunu","Rumble,Nidalee,Twisted Fate","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Singed,Warwick,Annie,Vayne,Janna","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Dyrus,Dan Dinh,Salce,Westrice,Doublelift",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Season 1 World Championship,Epik Gamer,against All authority,1,0,2011-06-18 08:30:00,45:09,45.150000,"Rumble,Nidalee,Twisted Fate","Akali,Vladimir,Nunu","Singed,Warwick,Annie,Vayne,Janna","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Dyrus,Dan Dinh,Salce,Westrice,Doublelift","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Season 1 World Championship,against All authority,Fnatic,1,0,2011-06-18 11:00:00,41:18,41.300000,"Alistar,Rumble,Twisted Fate","Irelia,Karthus,Nidalee","Gragas,Nunu,Anivia,Ashe,Sona","Vladimir,Amumu,Brand,Miss Fortune,Taric","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","wewillfailer,Cyanide,Shushei,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Season 1 World Championship,Fnatic,against All authority,0,1,2011-06-18 11:00:00,41:18,41.300000,"Irelia,Karthus,Nidalee","Alistar,Rumble,Twisted Fate","Vladimir,Amumu,Brand,Miss Fortune,Taric","Gragas,Nunu,Anivia,Ashe,Sona","wewillfailer,Cyanide,Shushei,Lamia,Mellisan","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Season 1 World Championship,against All authority,Fnatic,0,1,2011-06-20 22:00:00,43:57,43.950000,"Alistar,Gragas,Anivia","Rumble,Nunu,Amumu","Malzahar,Jarvan IV,Irelia,Corki,Janna","Brand,Warwick,Karthus,Ashe,Sona","MoMa,Linak,sOAZ,YellOwStaR,Kujaa","Shushei,Cyanide,xPeke,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33737,LEC/2022 Season/Summer,G2 Esports,Rogue (European Team),0,1,2022-09-11 17:17:00,29:09,29.150000,"Lucian,Renekton,Kalista,Caitlyn,Draven","Sylas,Senna,Yuumi,LeBlanc,Akali","Fiora,Trundle,Ahri,Zeri,Soraka","Aatrox,Jarvan IV,Azir,Sivir,Lulu","BrokenBlade,Jankos,caPs,Flakked,Targamas","Odoamne,Malrang,Larssen,Comp,Trymbi",1.0,3.0,0.0,1.0,3.0,10.0,47671.0,57510.0,3.0,12.0,0.0,2.0,0.0,2.0
33738,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,0,1,2022-08-26 16:13:00,38:03,38.050000,"Draven,Lulu,Gwen,Wukong,Vi","Azir,Poppy,Corki,Sivir,Ezreal","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Ornn,Trundle,Taliyah,Lucian,Nami","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",1.0,4.0,0.0,1.0,4.0,9.0,62217.0,70311.0,11.0,26.0,2.0,0.0,0.0,1.0
33739,LEC/2022 Season/Summer,MAD Lions,Rogue (European Team),1,0,2022-08-26 16:13:00,38:03,38.050000,"Azir,Poppy,Corki,Sivir,Ezreal","Draven,Lulu,Gwen,Wukong,Vi","Ornn,Trundle,Taliyah,Lucian,Nami","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser","Odoamne,Malrang,Larssen,Comp,Trymbi",4.0,1.0,1.0,0.0,9.0,4.0,70311.0,62217.0,26.0,11.0,0.0,2.0,1.0,0.0
33740,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,1,0,2022-08-26 19:13:00,27:29,27.483333,"Draven,Yuumi,Poppy,Nautilus,Amumu","Azir,Sivir,Renata Glasc,Wukong,Taliyah","Ornn,Jarvan IV,Ahri,Lucian,Nami","Gwen,Trundle,Swain,Kalista,Leona","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",3.0,0.0,1.0,0.0,10.0,3.0,56770.0,43836.0,25.0,8.0,1.0,1.0,1.0,0.0


In [101]:
games.to_csv('./csv/major_matches.csv', index=False)

In [102]:
WINNERS = {
    # Worlds
    'Season 1 World Championship': 'Fnatic',
    'Season 2 World Championship': 'Taipei Assassins',
    'Season 3 World Championship': 'SK Telecom T1',
    '2014 Season World Championship': 'Samsung White',
    '2015 Season World Championship': 'SK Telecom T1',
    '2016 Season World Championship': 'SK Telecom T1',
    '2017 Season World Championship': 'Samsung Galaxy',
    '2018 Season World Championship': 'Invictus Gaming',
    '2019 Season World Championship': 'FunPlus Phoenix',
    '2020 Season World Championship': 'DAMWON Gaming',
    '2021 Season World Championship': 'EDward Gaming',
    # MSI
    '2015 Mid-Season Invitational': 'EDward Gaming',
    '2016 Mid-Season Invitational': 'SK Telecom T1',
    '2017 Mid-Season Invitational': 'SK Telecom T1',
    '2018 Mid-Season Invitational': 'Royal Never Give Up',
    '2019 Mid-Season Invitational': 'G2 Esports',
    '2021 Mid-Season Invitational': 'Royal Never Give Up',
    '2022 Mid-Season Invitational': 'Royal Never Give Up',
    # LCK
    'Champions/2012 Season/Spring': 'MiG Blaze',
    'Champions/2012 Season/Summer': 'Azubu Frost',
    'Champions/2013 Season/Winter': 'NaJin Sword',
    'Champions/2013 Season/Spring': 'MVP Ozone',
    'Champions/2013 Season/Summer': 'SK Telecom T1 2',
    'Champions/2014 Season/Winter': 'SK Telecom T1 K',
    'Champions/2014 Season/Spring': 'Samsung Blue',
    'Champions/2014 Season/Summer': 'KT Rolster Arrows',
    'Champions/2015 Season/Spring': 'SK Telecom T1',
    'Champions/2015 Season/Summer': 'SK Telecom T1',
    'LCK/2016 Season/Spring': 'SK Telecom T1',
    'LCK/2016 Season/Summer': 'ROX Tigers',
    'LCK/2017 Season/Spring': 'SK Telecom T1',
    'LCK/2017 Season/Summer': 'Longzhu Gaming',
    'LCK/2018 Season/Spring': 'Kingzone DragonX',
    'LCK/2018 Season/Summer': 'KT Rolster',
    'LCK/2019 Season/Spring': 'SK Telecom T1',
    'LCK/2019 Season/Summer': 'SK Telecom T1',
    'LCK/2020 Season/Spring': 'T1',
    'LCK/2020 Season/Summer': 'DAMWON Gaming',
    'LCK/2021 Season/Spring': 'DWG KIA',
    'LCK/2021 Season/Summer': 'DWG KIA',
    'LCK/2022 Season/Spring': 'T1',
    'LCK/2022 Season/Summer': 'Gen.G',
    # LPL
    'LPL/2013 Season/Spring': 'Oh My God',
    'LPL/2013 Season/Summer': 'Positive Energy',
    'LPL/2014 Season/Spring': 'EDward Gaming',
    'LPL/2014 Season/Summer': 'EDward Gaming',
    'LPL/2015 Season/Spring': 'EDward Gaming',
    'LPL/2015 Season/Summer': 'LGD Gaming',
    'LPL/2016 Season/Spring': 'Royal Never Give Up',
    'LPL/2016 Season/Summer': 'EDward Gaming',
    'LPL/2017 Season/Spring': 'Team WE',
    'LPL/2017 Season/Summer': 'EDward Gaming',
    'LPL/2018 Season/Spring': 'Royal Never Give Up',
    'LPL/2018 Season/Summer': 'Royal Never Give Up',
    'LPL/2019 Season/Spring': 'Invictus Gaming',
    'LPL/2019 Season/Summer': 'FunPlus Phoenix',
    'LPL/2020 Season/Spring': 'JD Gaming',
    'LPL/2020 Season/Summer': 'Top Esports',
    'LPL/2021 Season/Spring': 'Royal Never Give Up',
    'LPL/2021 Season/Summer': 'EDward Gaming',
    'LPL/2022 Season/Spring': 'Royal Never Give Up',
    'LPL/2022 Season/Summer': 'JD Gaming',
    # LEC
    'EU LCS/Season 3/Spring': 'Fnatic',
    'EU LCS/Season 3/Summer': 'Fnatic',
    'EU LCS/2014 Season/Spring': 'Fnatic',
    'EU LCS/2014 Season/Summer': 'Alliance',
    'EU LCS/2015 Season/Spring': 'Fnatic',
    'EU LCS/2015 Season/Summer': 'Fnatic',
    'EU LCS/2016 Season/Spring': 'G2 Esports',
    'EU LCS/2016 Season/Summer': 'G2 Esports',
    'EU LCS/2017 Season/Spring': 'G2 Esports',
    'EU LCS/2017 Season/Summer': 'G2 Esports',
    'EU LCS/2018 Season/Spring': 'Fnatic',
    'EU LCS/2018 Season/Summer': 'Fnatic',
    'LEC/2019 Season/Spring': 'G2 Esports',
    'LEC/2019 Season/Summer': 'G2 Esports',
    'LEC/2020 Season/Spring': 'G2 Esports',
    'LEC/2020 Season/Summer': 'G2 Esports',
    'LEC/2021 Season/Spring': 'MAD Lions',
    'LEC/2021 Season/Summer': 'MAD Lions',
    'LEC/2022 Season/Spring': 'G2 Esports',
    'LEC/2022 Season/Summer': 'Rogue (European Team)',
    # LCS
    'NA LCS/Season 3/Spring': 'TSM',
    'NA LCS/Season 3/Summer': 'Cloud9',
    'NA LCS/2014 Season/Spring': 'Cloud9',
    'NA LCS/2014 Season/Summer': 'TSM',
    'NA LCS/2015 Season/Spring': 'TSM',
    'NA LCS/2015 Season/Summer': 'Counter Logic Gaming',
    'NA LCS/2016 Season/Spring': 'Counter Logic Gaming',
    'NA LCS/2016 Season/Summer': 'TSM',
    'NA LCS/2017 Season/Spring': 'TSM',
    'NA LCS/2017 Season/Summer': 'TSM',
    'NA LCS/2018 Season/Spring': 'Team Liquid',
    'NA LCS/2018 Season/Summer': 'Team Liquid',
    'LCS/2019 Season/Spring': 'Team Liquid',
    'LCS/2019 Season/Summer': 'Team Liquid',
    'LCS/2020 Season/Spring': 'Cloud9',
    'LCS/2020 Season/Summer': 'TSM',
    'LCS/2021 Season/Spring': 'Cloud9',
    'LCS/2021 Season/Summer': '100 Thieves',
    'LCS/2022 Season/Spring': 'Evil Geniuses.NA',
    'LCS/2022 Season/Summer': 'Cloud9',
}

In [106]:
winners = pd.DataFrame(data=WINNERS.items(), columns=['OverviewPage', 'Winner'])
winners

,OverviewPage,Winner
0,Season 1 World Championship,Fnatic
1,Season 2 World Championship,Taipei Assassins
2,Season 3 World Championship,SK Telecom T1
3,2014 Season World Championship,Samsung White
4,2015 Season World Championship,SK Telecom T1
...,...,...
97,LCS/2020 Season/Summer,TSM
98,LCS/2021 Season/Spring,Cloud9
99,LCS/2021 Season/Summer,100 Thieves
100,LCS/2022 Season/Spring,Evil Geniuses.NA


In [107]:
winners.to_csv('./csv/major_matches_winners.csv', index=False)

In [121]:
games = pd.read_csv('./csv/major_matches.csv')
games

,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,TeamBans,OpponentBans,TeamPicks,OpponentPicks,TeamPlayers,OpponentPlayers,TeamDragons,OpponentDragons,TeamBarons,OpponentBarons,TeamTowers,OpponentTowers,TeamGold,OpponentGold,TeamKills,OpponentKills,TeamRiftHeralds,OpponentHeralds,TeamInhibitors,OpponentInhibitors
0,Season 1 World Championship,against All authority,Epik Gamer,0,1,2011-06-18 08:30:00,45:09,45.150000,"Akali,Vladimir,Nunu","Rumble,Nidalee,Twisted Fate","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Singed,Warwick,Annie,Vayne,Janna","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Dyrus,Dan Dinh,Salce,Westrice,Doublelift",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Season 1 World Championship,Epik Gamer,against All authority,1,0,2011-06-18 08:30:00,45:09,45.150000,"Rumble,Nidalee,Twisted Fate","Akali,Vladimir,Nunu","Singed,Warwick,Annie,Vayne,Janna","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Dyrus,Dan Dinh,Salce,Westrice,Doublelift","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Season 1 World Championship,against All authority,Fnatic,1,0,2011-06-18 11:00:00,41:18,41.300000,"Alistar,Rumble,Twisted Fate","Irelia,Karthus,Nidalee","Gragas,Nunu,Anivia,Ashe,Sona","Vladimir,Amumu,Brand,Miss Fortune,Taric","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","wewillfailer,Cyanide,Shushei,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Season 1 World Championship,Fnatic,against All authority,0,1,2011-06-18 11:00:00,41:18,41.300000,"Irelia,Karthus,Nidalee","Alistar,Rumble,Twisted Fate","Vladimir,Amumu,Brand,Miss Fortune,Taric","Gragas,Nunu,Anivia,Ashe,Sona","wewillfailer,Cyanide,Shushei,Lamia,Mellisan","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Season 1 World Championship,against All authority,Fnatic,0,1,2011-06-20 22:00:00,43:57,43.950000,"Alistar,Gragas,Anivia","Rumble,Nunu,Amumu","Malzahar,Jarvan IV,Irelia,Corki,Janna","Brand,Warwick,Karthus,Ashe,Sona","MoMa,Linak,sOAZ,YellOwStaR,Kujaa","Shushei,Cyanide,xPeke,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33737,LEC/2022 Season/Summer,G2 Esports,Rogue (European Team),0,1,2022-09-11 17:17:00,29:09,29.150000,"Lucian,Renekton,Kalista,Caitlyn,Draven","Sylas,Senna,Yuumi,LeBlanc,Akali","Fiora,Trundle,Ahri,Zeri,Soraka","Aatrox,Jarvan IV,Azir,Sivir,Lulu","BrokenBlade,Jankos,caPs,Flakked,Targamas","Odoamne,Malrang,Larssen,Comp,Trymbi",1.0,3.0,0.0,1.0,3.0,10.0,47671.0,57510.0,3.0,12.0,0.0,2.0,0.0,2.0
33738,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,0,1,2022-08-26 16:13:00,38:03,38.050000,"Draven,Lulu,Gwen,Wukong,Vi","Azir,Poppy,Corki,Sivir,Ezreal","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Ornn,Trundle,Taliyah,Lucian,Nami","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",1.0,4.0,0.0,1.0,4.0,9.0,62217.0,70311.0,11.0,26.0,2.0,0.0,0.0,1.0
33739,LEC/2022 Season/Summer,MAD Lions,Rogue (European Team),1,0,2022-08-26 16:13:00,38:03,38.050000,"Azir,Poppy,Corki,Sivir,Ezreal","Draven,Lulu,Gwen,Wukong,Vi","Ornn,Trundle,Taliyah,Lucian,Nami","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser","Odoamne,Malrang,Larssen,Comp,Trymbi",4.0,1.0,1.0,0.0,9.0,4.0,70311.0,62217.0,26.0,11.0,0.0,2.0,1.0,0.0
33740,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,1,0,2022-08-26 19:13:00,27:29,27.483333,"Draven,Yuumi,Poppy,Nautilus,Amumu","Azir,Sivir,Renata Glasc,Wukong,Taliyah","Ornn,Jarvan IV,Ahri,Lucian,Nami","Gwen,Trundle,Swain,Kalista,Leona","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",3.0,0.0,1.0,0.0,10.0,3.0,56770.0,43836.0,25.0,8.0,1.0,1.0,1.0,0.0


In [122]:
winners = pd.read_csv('./csv/major_matches_winners.csv')
winners

,OverviewPage,Winner
0,Season 1 World Championship,Fnatic
1,Season 2 World Championship,Taipei Assassins
2,Season 3 World Championship,SK Telecom T1
3,2014 Season World Championship,Samsung White
4,2015 Season World Championship,SK Telecom T1
...,...,...
97,LCS/2020 Season/Summer,TSM
98,LCS/2021 Season/Spring,Cloud9
99,LCS/2021 Season/Summer,100 Thieves
100,LCS/2022 Season/Spring,Evil Geniuses.NA


In [123]:
games = pd.merge(games, winners, how='left', left_on=['OverviewPage', 'Team'], right_on=['OverviewPage', 'Winner'])
games['Winner'] = games['Winner'].apply(lambda x: 1 if isinstance(x, str) else 0)
games

,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,TeamBans,OpponentBans,TeamPicks,OpponentPicks,TeamPlayers,OpponentPlayers,TeamDragons,OpponentDragons,TeamBarons,OpponentBarons,TeamTowers,OpponentTowers,TeamGold,OpponentGold,TeamKills,OpponentKills,TeamRiftHeralds,OpponentHeralds,TeamInhibitors,OpponentInhibitors,Winner
0,Season 1 World Championship,against All authority,Epik Gamer,0,1,2011-06-18 08:30:00,45:09,45.150000,"Akali,Vladimir,Nunu","Rumble,Nidalee,Twisted Fate","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Singed,Warwick,Annie,Vayne,Janna","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","Dyrus,Dan Dinh,Salce,Westrice,Doublelift",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Season 1 World Championship,Epik Gamer,against All authority,1,0,2011-06-18 08:30:00,45:09,45.150000,"Rumble,Nidalee,Twisted Fate","Akali,Vladimir,Nunu","Singed,Warwick,Annie,Vayne,Janna","Irelia,Amumu,Karthus,Miss Fortune,Soraka","Dyrus,Dan Dinh,Salce,Westrice,Doublelift","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Season 1 World Championship,against All authority,Fnatic,1,0,2011-06-18 11:00:00,41:18,41.300000,"Alistar,Rumble,Twisted Fate","Irelia,Karthus,Nidalee","Gragas,Nunu,Anivia,Ashe,Sona","Vladimir,Amumu,Brand,Miss Fortune,Taric","sOAZ,Linak,MoMa,YellOwStaR,Kujaa","wewillfailer,Cyanide,Shushei,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Season 1 World Championship,Fnatic,against All authority,0,1,2011-06-18 11:00:00,41:18,41.300000,"Irelia,Karthus,Nidalee","Alistar,Rumble,Twisted Fate","Vladimir,Amumu,Brand,Miss Fortune,Taric","Gragas,Nunu,Anivia,Ashe,Sona","wewillfailer,Cyanide,Shushei,Lamia,Mellisan","sOAZ,Linak,MoMa,YellOwStaR,Kujaa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Season 1 World Championship,against All authority,Fnatic,0,1,2011-06-20 22:00:00,43:57,43.950000,"Alistar,Gragas,Anivia","Rumble,Nunu,Amumu","Malzahar,Jarvan IV,Irelia,Corki,Janna","Brand,Warwick,Karthus,Ashe,Sona","MoMa,Linak,sOAZ,YellOwStaR,Kujaa","Shushei,Cyanide,xPeke,Lamia,Mellisan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33737,LEC/2022 Season/Summer,G2 Esports,Rogue (European Team),0,1,2022-09-11 17:17:00,29:09,29.150000,"Lucian,Renekton,Kalista,Caitlyn,Draven","Sylas,Senna,Yuumi,LeBlanc,Akali","Fiora,Trundle,Ahri,Zeri,Soraka","Aatrox,Jarvan IV,Azir,Sivir,Lulu","BrokenBlade,Jankos,caPs,Flakked,Targamas","Odoamne,Malrang,Larssen,Comp,Trymbi",1.0,3.0,0.0,1.0,3.0,10.0,47671.0,57510.0,3.0,12.0,0.0,2.0,0.0,2.0,0
33738,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,0,1,2022-08-26 16:13:00,38:03,38.050000,"Draven,Lulu,Gwen,Wukong,Vi","Azir,Poppy,Corki,Sivir,Ezreal","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Ornn,Trundle,Taliyah,Lucian,Nami","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",1.0,4.0,0.0,1.0,4.0,9.0,62217.0,70311.0,11.0,26.0,2.0,0.0,0.0,1.0,1
33739,LEC/2022 Season/Summer,MAD Lions,Rogue (European Team),1,0,2022-08-26 16:13:00,38:03,38.050000,"Azir,Poppy,Corki,Sivir,Ezreal","Draven,Lulu,Gwen,Wukong,Vi","Ornn,Trundle,Taliyah,Lucian,Nami","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser","Odoamne,Malrang,Larssen,Comp,Trymbi",4.0,1.0,1.0,0.0,9.0,4.0,70311.0,62217.0,26.0,11.0,0.0,2.0,1.0,0.0,0
33740,LEC/2022 Season/Summer,Rogue (European Team),MAD Lions,1,0,2022-08-26 19:13:00,27:29,27.483333,"Draven,Yuumi,Poppy,Nautilus,Amumu","Azir,Sivir,Renata Glasc,Wukong,Taliyah","Ornn,Jarvan IV,Ahri,Lucian,Nami","Gwen,Trundle,Swain,Kalista,Leona","Odoamne,Malrang,Larssen,Comp,Trymbi","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",3.0,0.0,1.0,0.0,10.0,3.0,56770.0,43836.0,25.0,8.0,1.0,1.0,1.0,0.0,1


In [62]:
leagues = get_leagues(where='L.League_Short in ("LTC")')
# leagues = get_leagues(where='L.Region="Korea"')
leagues

,League,League Short,Region,Level,IsOfficial
0,LoL The Champions,LTC,Korea,Primary,Yes


In [44]:
tournaments = pd.DataFrame()
for league in leagues['League Short']:
    # tournament = get_tournaments(where=f'L.League_Short="{league}"')
    # tournament = get_tournaments(where=f'L.League_Short="{league}" and T.Region="International"')
    tournament = get_tournaments(where=f'L.League_Short="{league}" and T.IsQualifier=0')
    tournaments = pd.concat([tournaments, tournament])
    
tournaments = tournaments.sort_values(by=['Year', 'DateStart', 'Date']).reset_index(drop=True)
tournaments.head()

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,NA LCS 2013 Spring,NA LCS/Season 3/Spring Season,2013-02-07,2013-04-19,North America League Championship Series,North America,Offline,NA LCS 2013 Spring,Spring,1,Primary,0,0,1,2013,1,1
1,NA LCS 2013 Spring Playoffs,NA LCS/Season 3/Spring Playoffs,2013-04-26,2013-04-28,North America League Championship Series,North America,Offline,NA LCS 2013 Spring Playoffs,Spring,1,Primary,0,1,1,2013,1,1
2,NA LCS 2013 Summer,NA LCS/Season 3/Summer Season,2013-06-12,2013-08-16,North America League Championship Series,North America,Offline,NA LCS 2013 Summer,Summer,2,Primary,0,0,1,2013,1,1
3,NA LCS 2013 Summer Playoffs,NA LCS/Season 3/Summer Playoffs,2013-08-30,2013-09-01,North America League Championship Series,North America,Offline,NA LCS 2013 Summer Playoffs,Summer,None,Primary,0,1,1,2013,1,1
4,NA LCS 2014 Spring,NA LCS/2014 Season/Spring Season,2014-01-16,2014-04-06,North America League Championship Series,North America,Offline,NA LCS 2014 Spring,Spring,1,Primary,0,0,1,2014,1,1


In [45]:
tournaments['OverviewPage'].unique()

array(['NA LCS/Season 3/Spring Season', 'NA LCS/Season 3/Spring Playoffs',
       'NA LCS/Season 3/Summer Season', 'NA LCS/Season 3/Summer Playoffs',
       'NA LCS/2014 Season/Spring Season',
       'NA LCS/2014 Season/Spring Playoffs',
       'NA LCS/2014 Season/Summer Season',
       'NA LCS/2014 Season/Summer Playoffs',
       'NA LCS/2015 Season/Spring Season',
       'NA LCS/2015 Season/Spring Playoffs',
       'NA LCS/2015 Season/Summer Season',
       'NA LCS/2015 Season/Summer Playoffs',
       'NA LCS/2016 Season/Spring Season',
       'NA LCS/2016 Season/Spring Playoffs',
       'NA LCS/2016 Season/Summer Season',
       'NA LCS/2016 Season/Summer Playoffs',
       'NA LCS/2017 Season/Spring Season',
       'NA LCS/2017 Season/Spring Playoffs',
       'NA LCS/2017 Season/Summer Season',
       'NA LCS/2017 Season/Summer Playoffs',
       'NA LCS/2018 Season/Spring Season',
       'NA LCS/2018 Season/Spring Playoffs',
       'NA LCS/2018 Season/Summer Season',
       'NA LCS/

In [56]:
i = 35
overviews = '"' + tournaments.iloc[i]['OverviewPage'] + '"'
scoreboard_games = get_scoreboard_games(where=f'T.OverviewPage in ({overviews})')
if scoreboard_games is None:
    print(f'{tournaments.iloc[i]["OverviewPage"]} is None')
print(scoreboard_games.shape)
print(scoreboard_games['OverviewPage'].unique())
scoreboard_games.head()

(31, 36)
['LCS/2021 Season/Mid-Season Showdown']


,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,LCS/2021 Season/Mid-Season Showdown,100 Thieves,Cloud9,Cloud9,100 Thieves,2021-03-21 20:42:00,0,2,2,30:15,30.250000,"Seraphine,Tristana,Lillia,Nautilus,Kalista","Hecarim,Renekton,Rell,Gnar,Camille","Aatrox,Udyr,Azir,Jinx,Thresh","Sion,Olaf,Orianna,Senna,Tahm Kench","Ssumday,Closer (Can Çelik),ry0ma,FBI,huhi","Fudge,Blaber,Perkz,Zven,Vulcan (Philippe Lafla...",0.0,5.0,1.0,0.0,3.0,8.0,47703.0,57086.0,3.0,16.0,0.0,2.0,0.0,1.0,11.5,LCS/2021 Season/Mid-Season Showdown_Round 1_2_2,LCS/2021 Season/Mid-Season Showdown_Round 1_2,1725497,0
1,LCS/2021 Season/Mid-Season Showdown,100 Thieves,Cloud9,Cloud9,100 Thieves,2021-03-21 21:41:00,0,3,2,31:30,31.500000,"Sion,Olaf,Seraphine,Irelia,Renekton","Hecarim,Rell,Nidalee,Lucian,Gnar","Cho'Gath,Udyr,Syndra,Senna,Tahm Kench","Sett,Lillia,Tristana,Ezreal,Karma","Ssumday,Closer (Can Çelik),ry0ma,FBI,huhi","Fudge,Blaber,Perkz,Zven,Vulcan (Philippe Lafla...",2.0,2.0,0.0,1.0,2.0,8.0,50151.0,59749.0,8.0,13.0,1.0,1.0,0.0,1.0,11.5,LCS/2021 Season/Mid-Season Showdown_Round 1_2_3,LCS/2021 Season/Mid-Season Showdown_Round 1_2,1725516,0
2,LCS/2021 Season/Mid-Season Showdown,100 Thieves,Dignitas,100 Thieves,Dignitas,2021-03-28 20:39:00,2,0,1,36:07,36.116667,"Seraphine,Lillia,Kalista,Jinx,Twisted Fate","Tahm Kench,Tristana,Renekton,Nautilus,Sett","Gnar,Hecarim,Orianna,Kai'Sa,Rell","Gangplank,Udyr,Ahri,Xayah,Thresh","Ssumday,Closer (Can Çelik),ry0ma,FBI,huhi","FakeGod,Dardoch,Soligo,Neo (Toàn Trần),aphromoo",4.0,1.0,1.0,0.0,11.0,2.0,72424.0,57031.0,20.0,7.0,2.0,0.0,2.0,0.0,11.5,LCS/2021 Season/Mid-Season Showdown_Round 1_4_2,LCS/2021 Season/Mid-Season Showdown_Round 1_4,1760502,0
3,LCS/2021 Season/Mid-Season Showdown,100 Thieves,Dignitas,100 Thieves,Dignitas,2021-03-28 19:25:00,1,0,1,36:12,36.200000,"Seraphine,Lillia,Nidalee,Udyr,Twisted Fate","Tahm Kench,Tristana,Renekton,Nautilus,Sett","Gnar,Hecarim,Orianna,Kai'Sa,Alistar","Gangplank,Volibear,Ahri,Kalista,Thresh","Ssumday,Closer (Can Çelik),ry0ma,FBI,huhi","FakeGod,Dardoch,Soligo,Neo (Toàn Trần),aphromoo",2.0,3.0,1.0,0.0,11.0,3.0,71310.0,60779.0,24.0,12.0,1.0,1.0,3.0,0.0,11.5,LCS/2021 Season/Mid-Season Showdown_Round 1_4_1,LCS/2021 Season/Mid-Season Showdown_Round 1_4,1760500,0
4,LCS/2021 Season/Mid-Season Showdown,100 Thieves,TSM,100 Thieves,TSM,2021-04-04 21:50:00,1,2,1,36:04,36.066667,"Seraphine,Rell,Tristana,Thresh,Twisted Fate","Senna,Hecarim,Renekton,Kalista,Kai'Sa","Jayce,Udyr,Orianna,Ezreal,Leona","Gnar,Nidalee,Azir,Jinx,Alistar","Ssumday,Closer (Can Çelik),ry0ma,FBI,huhi","Huni,Spica,PowerOfEvil,Lost,SwordArt",5.0,0.0,0.0,1.0,8.0,5.0,66895.0,60855.0,21.0,9.0,0.0,2.0,1.0,0.0,11.5,LCS/2021 Season/Mid-Season Showdown_Round 2_2_3,LCS/2021 Season/Mid-Season Showdown_Round 2_2,1800225,0


In [245]:
i = 11
j = 1
overviews = ', '.join(map(lambda x: '"' + x + '"', tournaments.iloc[[i, j]]['OverviewPage']))
# overviews = ', '.join(map(lambda x: '"' + x + '"', tournaments.iloc[i:j+1]['OverviewPage']))
print(overviews)
scoreboard_games = get_scoreboard_games(where=f'T.OverviewPage in ({overviews})')
if scoreboard_games is None:
    print(f'{tournaments.iloc[i]["OverviewPage"]} is None')
# scoreboard_games['OverviewPage'] = scoreboard_games['OverviewPage'].str.split(' ').apply(lambda x: ' '.join(x[:-1]))
scoreboard_games['OverviewPage'] = scoreboard_games['OverviewPage'].str.split('/').apply(lambda x: x[0])
print(scoreboard_games.shape)
print(scoreboard_games['OverviewPage'].unique())
scoreboard_games.head()

"2019 Mid-Season Invitational/Play-In", "2019 Mid-Season Invitational/Main Event"
(79, 36)
['2019 Mid-Season Invitational']


,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,2019 Mid-Season Invitational,Flash Wolves,G2 Esports,G2 Esports,Flash Wolves,2019-05-11 10:44:00,0,1,2,32:22,32.366667,"Taric,Irelia,Jayce,Olaf,Karthus","Rek'Sai,Akali,Nautilus,Kalista,Kai'Sa","Ryze,Jarvan IV,Orianna,Lucian,Galio","Neeko,Lee Sin,Sylas,Ezreal,Tahm Kench","Hanabi (Su Chia-Hsiang),Bugi,Rather,Betty,ShiauC","Wunder,Jankos,Caps,Perkz,Mikyx",2.0,3.0,0.0,2.0,0.0,11.0,56947.0,67974.0,12.0,16.0,1.0,0.0,0.0,5.0,9.8,2019 Mid-Season Invitational/Main Event_Day 2_5_1,2019 Mid-Season Invitational/Main Event_Day 2_5,1121178,0
1,2019 Mid-Season Invitational,Flash Wolves,Invictus Gaming,Invictus Gaming,Flash Wolves,2019-05-12 07:45:00,0,1,2,44:03,44.050000,"Irelia,Jayce,Sylas,LeBlanc,Zoe","Jarvan IV,Rek'Sai,Tahm Kench,Varus,Kalista","Akali,Xin Zhao,Orianna,Ezreal,Braum","Kennen,Kayn,Ryze,Kai'Sa,Galio","Hanabi (Su Chia-Hsiang),Bugi,Rather,Betty,ShiauC","TheShy,Ning,Rookie,JackeyLove,Baolan",3.0,3.0,0.0,3.0,4.0,11.0,78948.0,87334.0,20.0,17.0,1.0,0.0,0.0,3.0,9.8,2019 Mid-Season Invitational/Main Event_Day 3_2_1,2019 Mid-Season Invitational/Main Event_Day 3_2,1121263,0
2,2019 Mid-Season Invitational,Flash Wolves,Phong Vũ Buffalo,Flash Wolves,Phong Vũ Buffalo,2019-05-14 13:27:00,1,0,1,30:47,30.783333,"Jayce,Akali,Rek'Sai,Hecarim,Fiora","Tahm Kench,Galio,Nautilus,Orianna,Gangplank","Yasuo,Evelynn,Talon,Kalista,Thresh","Irelia,Jarvan IV,Sylas,Varus,Morgana","Hanabi (Su Chia-Hsiang),Bugi,Rather,Betty,ShiauC","Zeros,Meliodas (Hoàng Tiến Nhật),Naul,BigKoro,...",2.0,2.0,1.0,0.0,8.0,4.0,59040.0,51612.0,18.0,10.0,0.0,1.0,1.0,0.0,9.8,2019 Mid-Season Invitational/Main Event_Day 5_5_1,2019 Mid-Season Invitational/Main Event_Day 5_5,1121475,0
3,2019 Mid-Season Invitational,Flash Wolves,SK Telecom T1,SK Telecom T1,Flash Wolves,2019-05-13 12:46:00,0,1,2,31:11,31.183333,"Jayce,Irelia,Lee Sin,LeBlanc,Kennen","Rek'Sai,Ryze,Jarvan IV,Nautilus,Braum","Akali,Xin Zhao,Orianna,Varus,Tahm Kench","Sylas,Vi,Azir,Kalista,Galio","Hanabi (Su Chia-Hsiang),Bugi,Rather,Betty,ShiauC","Khan,Clid,Faker,Teddy,Mata",0.0,5.0,0.0,2.0,0.0,11.0,46250.0,67202.0,2.0,16.0,0.0,1.0,0.0,2.0,9.8,2019 Mid-Season Invitational/Main Event_Day 4_4_1,2019 Mid-Season Invitational/Main Event_Day 4_4,1121404,0
4,2019 Mid-Season Invitational,Flash Wolves,Team Liquid,Team Liquid,Flash Wolves,2019-05-10 10:41:00,0,1,2,28:02,28.033333,"Taric,Tahm Kench,Jayce,Kennen,Sejuani","Sylas,Rek'Sai,Jarvan IV,Ryze,LeBlanc","Vladimir,Nocturne,Azir,Kai'Sa,Galio","Gangplank,Trundle,Akali,Lucian,Braum","Hanabi (Su Chia-Hsiang),Bugi,Rather,Betty,ShiauC","Impact,Xmithie,Jensen,Doublelift,CoreJJ",0.0,4.0,0.0,1.0,0.0,8.0,42792.0,54748.0,1.0,11.0,0.0,1.0,0.0,1.0,9.8,2019 Mid-Season Invitational/Main Event_Day 1_2_1,2019 Mid-Season Invitational/Main Event_Day 1_2,1120997,0


In [246]:
# scoreboard_games['OverviewPage'] = ['LCS/2022 Season/Summer'] * scoreboard_games.shape[0]
# scoreboard_games['OverviewPage'].unique()

In [262]:
games = change_data(scoreboard_games)
print(scoreboard_games.shape[0] * 2 == games.shape[0])
games.head()

True


,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,Gold,Kills,Deaths,KPM,KDPM,GPM,GDPM
0,2022 Mid-Season Invitational,DetonatioN FocusMe,Saigon Buffalo,0,1,2022-05-11 10:29:00,35:49,35.816667,64970.0,17.0,17.0,0.474639,0.000000,1813.959981,-95.877152
1,2022 Mid-Season Invitational,Saigon Buffalo,DetonatioN FocusMe,1,0,2022-05-11 10:29:00,35:49,35.816667,68404.0,17.0,17.0,0.474639,0.000000,1909.837134,95.877152
2,2022 Mid-Season Invitational,DetonatioN FocusMe,T1,0,1,2022-05-15 07:00:00,23:43,23.716667,38877.0,6.0,18.0,0.252987,-0.505973,1639.226985,-480.042164
3,2022 Mid-Season Invitational,T1,DetonatioN FocusMe,1,0,2022-05-15 07:00:00,23:43,23.716667,50262.0,18.0,6.0,0.758960,0.505973,2119.269150,480.042164
4,2022 Mid-Season Invitational,DetonatioN FocusMe,Team Aze,0,1,2022-05-15 09:36:00,37:03,37.050000,70590.0,19.0,16.0,0.512821,0.080972,1905.263158,-19.244265


In [263]:
print(games['OverviewPage'].unique())
games['Team'].unique()

['2022 Mid-Season Invitational']


array(['DetonatioN FocusMe', 'Saigon Buffalo', 'T1', 'Team Aze',
       'Evil Geniuses.NA', 'G2 Esports', 'ORDER', 'PSG Talon',
       'Royal Never Give Up', 'Istanbul Wildcats', 'RED Canids'],
      dtype=object)

In [265]:
teams = get_team_stats(games)
teams.loc[winners[teams['OverviewPage'][0]], 'Winner'] = 1
teams.head()

,OverviewPage,Games,Win,Loss,WinRate,Gamelength Number,Win Gamelength Number,Loss Gamelength Number,Gold,Kills,GPM,GDPM,KPM,KDPM,Winner
Team,,,,,,,,,,,,,,,
DetonatioN FocusMe,2022 Mid-Season Invitational,6,1,5,0.166667,30.791667,26.583333,31.633333,55967.166667,16.500000,1800.552233,-149.246387,0.513355,-0.087703,0
Evil Geniuses.NA,2022 Mid-Season Invitational,21,9,12,0.428571,31.973810,29.668519,33.702778,58267.904762,14.476190,1829.831116,23.091799,0.465180,0.025049,0
G2 Esports,2022 Mid-Season Invitational,21,13,8,0.619048,29.820635,31.802564,26.600000,55447.333333,14.904762,1855.417222,51.445981,0.500803,0.077364,0
Istanbul Wildcats,2022 Mid-Season Invitational,6,1,5,0.166667,27.238889,28.366667,27.013333,45865.500000,10.333333,1679.837707,-328.859628,0.373681,-0.481469,0
ORDER,2022 Mid-Season Invitational,8,0,8,0.000000,26.341667,NaN,26.341667,41050.500000,6.000000,1557.139621,-559.517561,0.230972,-0.556492,0


In [266]:
# team_stats = pd.DataFrame()
team_stats = pd.concat([team_stats, teams])
team_stats

,OverviewPage,Games,Win,Loss,WinRate,Gamelength Number,Win Gamelength Number,Loss Gamelength Number,Gold,Kills,GPM,GDPM,KPM,KDPM,Winner
Team,,,,,,,,,,,,,,,
Beşiktaş Esports,2015 Mid-Season Invitational,5,0,5,0.000000,27.583333,NaN,27.583333,36939.800000,5.400000,1318.179061,-694.737158,0.176694,-0.637776,0
EDward Gaming,2015 Mid-Season Invitational,13,10,3,0.769231,33.133333,32.218333,36.183333,59623.076923,22.846154,1823.463395,322.888397,0.727687,0.379631,1
Fnatic,2015 Mid-Season Invitational,10,4,6,0.400000,34.705000,32.925000,35.891667,56630.000000,16.100000,1637.845745,-28.081297,0.469567,0.006436,0
SK Telecom T1,2015 Mid-Season Invitational,15,10,5,0.666667,35.664444,35.776667,35.440000,59680.000000,15.866667,1678.774193,78.089968,0.453957,0.013543,0
TSM,2015 Mid-Season Invitational,5,1,4,0.200000,28.820000,26.416667,29.420833,42500.000000,7.000000,1482.208429,-313.740496,0.249994,-0.372266,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RED Canids,2022 Mid-Season Invitational,6,2,4,0.333333,27.391667,29.000000,26.587500,47600.666667,12.166667,1722.352848,-141.798372,0.424337,-0.105424,0
Royal Never Give Up,2022 Mid-Season Invitational,24,20,4,0.833333,30.121528,29.896667,31.245833,59150.458333,18.791667,1975.927723,300.646495,0.651153,0.334288,1
Saigon Buffalo,2022 Mid-Season Invitational,16,6,10,0.375000,28.847917,28.483333,29.066667,52122.250000,14.312500,1809.985821,-105.406974,0.503435,-0.126003,0


In [267]:
team_stats.reset_index().to_csv('./csv/msi_teams.csv', index=False)

In [1643]:
team_stats = pd.read_csv('./csv/msi_teams.csv')
team_stats

,Team,OverviewPage,Games,Win,Loss,WinRate,Gamelength Number,Win Gamelength Number,Loss Gamelength Number,Gold,Kills,GPM,GDPM,KPM,Winner
0,Beşiktaş Esports,2015 Mid-Season Invitational,5,0,5,0.000000,27.583333,NaN,27.583333,36939.800000,5.400000,1318.179061,-694.737158,0.176694,0
1,EDward Gaming,2015 Mid-Season Invitational,13,10,3,0.769231,33.133333,32.218333,36.183333,59623.076923,22.846154,1823.463395,322.888397,0.727687,1
2,Fnatic,2015 Mid-Season Invitational,10,4,6,0.400000,34.705000,32.925000,35.891667,56630.000000,16.100000,1637.845745,-28.081297,0.469567,0
3,SK Telecom T1,2015 Mid-Season Invitational,15,10,5,0.666667,35.664444,35.776667,35.440000,59680.000000,15.866667,1678.774193,78.089968,0.453957,0
4,TSM,2015 Mid-Season Invitational,5,1,4,0.200000,28.820000,26.416667,29.420833,42500.000000,7.000000,1482.208429,-313.740496,0.249994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,RED Canids,2022 Mid-Season Invitational,6,2,4,0.333333,27.391667,29.000000,26.587500,47600.666667,12.166667,1722.352848,-141.798372,0.424337,0
70,Royal Never Give Up,2022 Mid-Season Invitational,24,20,4,0.833333,30.121528,29.896667,31.245833,59150.458333,18.791667,1975.927723,300.646495,0.651153,1
71,Saigon Buffalo,2022 Mid-Season Invitational,16,6,10,0.375000,28.847917,28.483333,29.066667,52122.250000,14.312500,1809.985821,-105.406974,0.503435,0
72,T1,2022 Mid-Season Invitational,24,18,6,0.750000,27.369444,25.880556,31.836111,54169.833333,15.500000,2005.620210,315.283273,0.609463,0


In [6]:
league = get_leagues(where=f'L.League_Short="WCS"')
league

,League,League Short,Region,Level,IsOfficial
0,World Championship,WCS,International,Primary,Yes


In [7]:
tournaments = get_tournaments(where=f'L.League_Short="WCS" and T.Region="International"')
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,Worlds 2014,2014 Season World Championship,2014-09-18,2014-10-19,World Championship,International,Offline,Worlds 2014,None,None,Primary,0,0,1,2014,1,1
1,Worlds 2015,2015 Season World Championship,2015-10-01,2015-10-31,World Championship,International,Offline,Worlds 2015,None,None,Primary,0,0,1,2015,1,1
2,Worlds 2016,2016 Season World Championship,2016-09-29,2016-10-29,World Championship,International,Offline,Worlds 2016,None,None,Primary,0,0,1,2016,1,1
3,Worlds 2017,2017 Season World Championship,2017-09-23,2017-11-04,World Championship,International,Offline,None,None,None,None,None,None,None,None,1,1
4,Worlds 2017 Main Event,2017 Season World Championship/Main Event,2017-10-05,2017-11-04,World Championship,International,Offline,Worlds 2017 Main Event,None,None,Primary,0,0,1,2017,1,1
5,Worlds 2017 Play-In,2017 Season World Championship/Play-In,2017-09-23,2017-09-29,World Championship,International,Offline,Worlds 2017 Play-In,None,None,Primary,1,0,1,2017,1,1
6,Worlds 2018,2018 Season World Championship,2018-10-01,2018-11-03,World Championship,International,Offline,None,None,None,None,None,None,None,None,1,1
7,Worlds 2018 Main Event,2018 Season World Championship/Main Event,2018-10-10,2018-11-03,World Championship,International,Offline,Worlds 2018 Main Event,None,None,Primary,0,0,1,2018,1,1
8,Worlds 2018 Play-In,2018 Season World Championship/Play-In,2018-10-01,2018-10-07,World Championship,International,Offline,Worlds 2018 Play-In,None,None,Primary,1,0,1,2018,1,1
9,Worlds 2019,2019 Season World Championship,2019-10-02,2019-11-10,World Championship,International,Offline,None,None,None,None,None,None,None,None,1,1


In [8]:
tournaments = tournaments.loc[[19, 20]]
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
19,Worlds 2022 Main Event,2022 Season World Championship/Main Event,2022-10-07,2022-11-05,World Championship,International,Offline,Worlds 2022 Main Event,None,None,Primary,0,0,1,2022,1,1
20,Worlds 2022 Play-In,2022 Season World Championship/Play-In,2022-09-29,2022-10-04,World Championship,International,Offline,Worlds 2022 Play-In,None,None,Primary,1,0,1,2022,1,1


In [9]:
scoreboard_games = pd.DataFrame()
for overview in tournaments['OverviewPage']:
    temp = get_scoreboard_games(where=f'T.OverviewPage="{overview}"')
    scoreboard_games = pd.concat([scoreboard_games, temp])
scoreboard_games = scoreboard_games.reset_index(drop=True)
scoreboard_games['OverviewPage'] = '2022 Season World Championship'
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,2022 Season World Championship,100 Thieves,CTBC Flying Oyster,100 Thieves,CTBC Flying Oyster,2022-10-16 19:18:00,1,0,1,31:40,31.666667,"Viego,Fiora,Maokai,Renekton,Poppy","Aatrox,Sylas,Yuumi,Miss Fortune,Karma","Sejuani,Bel'Veth,Azir,Kalista,Amumu","Gnar,Jarvan IV,Viktor,Caitlyn,Lux","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Rest,Gemini,Mission,Shunn,Koala (Lin Chih-Chiang)",4.0,0.0,1.0,0.0,10.0,3.0,61615.0,50932.0,20.0,4.0,2.0,0.0,3.0,0.0,12.18,2022 Season World Championship/Main Event_Day ...,2022 Season World Championship/Main Event_Day 8_1,None,0
1,2022 Season World Championship,100 Thieves,Gen.G,Gen.G,100 Thieves,2022-10-09 22:18:00,0,1,2,29:30,29.500000,"Sejuani,Caitlyn,Yuumi,Lucian,Nilah","Graves,Kalista,Maokai,Varus,Nautilus","Aatrox,Poppy,Taliyah,Miss Fortune,Leona","Camille,Trundle,Azir,Aphelios,Renata Glasc","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...",0.0,4.0,0.0,1.0,1.0,9.0,45667.0,56529.0,8.0,17.0,2.0,0.0,0.0,2.0,12.18,2022 Season World Championship/Main Event_Day ...,2022 Season World Championship/Main Event_Day 3_2,None,0
2,2022 Season World Championship,100 Thieves,Royal Never Give Up,Royal Never Give Up,100 Thieves,2022-10-11 00:09:00,0,1,2,26:56,26.933333,"Fiora,Lissandra,Galio,Lucian,Varus","Yuumi,Maokai,Azir,LeBlanc,Sylas","Aatrox,Viego,Taliyah,Aphelios,Leona","Jax,Graves,Akali,Kai'Sa,Amumu","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Breathe,Wei (Yan Yang-Wei),Xiaohu,GALA,Ming (S...",0.0,4.0,0.0,1.0,3.0,8.0,42890.0,54884.0,4.0,17.0,2.0,0.0,0.0,1.0,12.18,2022 Season World Championship/Main Event_Day ...,2022 Season World Championship/Main Event_Day 4_4,None,0
3,2022 Season World Championship,Cloud9,EDward Gaming,EDward Gaming,Cloud9,2022-10-13 21:00:00,0,1,2,26:07,26.116667,"Caitlyn,Graves,Sylas,Lucian,Kai'Sa","Yuumi,Azir,Sejuani,Kalista,Jinx","Aatrox,Trundle,Viktor,Varus,Renata Glasc","Fiora,Maokai,Akali,Aphelios,Thresh","Fudge,Blaber,Jensen,Berserker (Kim Min-cheol),...","Flandre,JieJie,Scout,Viper (Park Do-hyeon),Meiko",0.0,3.0,0.0,1.0,0.0,9.0,36128.0,54394.0,1.0,19.0,0.0,2.0,0.0,2.0,12.18,2022 Season World Championship/Main Event_Day ...,2022 Season World Championship/Main Event_Day 5_3,None,0
4,2022 Season World Championship,Cloud9,Fnatic,Fnatic,Cloud9,2022-10-07 21:16:00,0,1,2,32:24,32.400000,"Graves,Trundle,Caitlyn,Leona,Renekton","Aatrox,Olaf,Yuumi,Sylas,Viktor","Fiora,Sejuani,LeBlanc,Miss Fortune,Nautilus","Ornn,Maokai,Azir,Aphelios,Renata Glasc","Fudge,Blaber,Jensen,Berserker (Kim Min-cheol),...","Wunder,Razork,Humanoid,Upset,Hylissang",1.0,4.0,0.0,1.0,3.0,9.0,52116.0,60456.0,6.0,15.0,2.0,0.0,0.0,1.0,12.18,2022 Season World Championship/Main Event_Day ...,2022 Season World Championship/Main Event_Day 1_1,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022 Season World Championship,Royal Never Give Up,Saigon Buffalo,Royal Never Give Up,Saigon Buffalo,2022-10-03 01:09:00,1,0,1,32:60,33.000000,"Pantheon,Kalista,Poppy,Kennen,Renata Glasc","Aatrox,Caitlyn,Graves,Kai'Sa,Xayah","Fiora,Vi,Sylas,Tristana,Shen","Lillia,Lee Sin,Zac,Aphelios,Leona","Breathe,Wei (Yan Yang-Wei),Xiaohu,GALA,Ming (S...","Hasmed,BeanJ,Froggy,Shogun (Nguyễn Văn Huy),Ta...",3.0,2.0,1.0,0.0,10.0,2.0,63454.0,53434.0,9.0,9.0,1.0,1.0,2.0,0.0,12.18,2022 Season World Championship/Play-In_Day 4_6_1,2022 Season World Championship/Play-In_Day 4_6,None,0
105,2022 Season World Championship,Saigon Buffalo,Istanbul Wildcats,Saigon Buffalo,Istanbul Wildcat

In [10]:
games = change_data(scoreboard_games)
print(scoreboard_games.shape[0] * 2 == games.shape[0])
games

True


,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,TeamBans,OpponentBans,TeamPicks,OpponentPicks,TeamPlayers,OpponentPlayers,TeamDragons,OpponentDragons,TeamBarons,OpponentBarons,TeamTowers,OpponentTowers,TeamGold,OpponentGold,TeamKills,OpponentKills,TeamRiftHeralds,OpponentHeralds,TeamInhibitors,OpponentInhibitors
0,2022 Season World Championship,100 Thieves,CTBC Flying Oyster,1,0,2022-10-16 19:18:00,31:40,31.666667,"Viego,Fiora,Maokai,Renekton,Poppy","Aatrox,Sylas,Yuumi,Miss Fortune,Karma","Sejuani,Bel'Veth,Azir,Kalista,Amumu","Gnar,Jarvan IV,Viktor,Caitlyn,Lux","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Rest,Gemini,Mission,Shunn,Koala (Lin Chih-Chiang)",4.0,0.0,1.0,0.0,10.0,3.0,61615.0,50932.0,20.0,4.0,2.0,0.0,3.0,0.0
1,2022 Season World Championship,CTBC Flying Oyster,100 Thieves,0,1,2022-10-16 19:18:00,31:40,31.666667,"Aatrox,Sylas,Yuumi,Miss Fortune,Karma","Viego,Fiora,Maokai,Renekton,Poppy","Gnar,Jarvan IV,Viktor,Caitlyn,Lux","Sejuani,Bel'Veth,Azir,Kalista,Amumu","Rest,Gemini,Mission,Shunn,Koala (Lin Chih-Chiang)","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi",0.0,4.0,0.0,1.0,3.0,10.0,50932.0,61615.0,4.0,20.0,0.0,2.0,0.0,3.0
2,2022 Season World Championship,100 Thieves,Gen.G,0,1,2022-10-09 22:18:00,29:30,29.500000,"Sejuani,Caitlyn,Yuumi,Lucian,Nilah","Graves,Kalista,Maokai,Varus,Nautilus","Aatrox,Poppy,Taliyah,Miss Fortune,Leona","Camille,Trundle,Azir,Aphelios,Renata Glasc","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...",0.0,4.0,0.0,1.0,1.0,9.0,45667.0,56529.0,8.0,17.0,2.0,0.0,0.0,2.0
3,2022 Season World Championship,Gen.G,100 Thieves,1,0,2022-10-09 22:18:00,29:30,29.500000,"Graves,Kalista,Maokai,Varus,Nautilus","Sejuani,Caitlyn,Yuumi,Lucian,Nilah","Camille,Trundle,Azir,Aphelios,Renata Glasc","Aatrox,Poppy,Taliyah,Miss Fortune,Leona","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi",4.0,0.0,1.0,0.0,9.0,1.0,56529.0,45667.0,17.0,8.0,0.0,2.0,2.0,0.0
4,2022 Season World Championship,100 Thieves,Royal Never Give Up,0,1,2022-10-11 00:09:00,26:56,26.933333,"Fiora,Lissandra,Galio,Lucian,Varus","Yuumi,Maokai,Azir,LeBlanc,Sylas","Aatrox,Viego,Taliyah,Aphelios,Leona","Jax,Graves,Akali,Kai'Sa,Amumu","Ssumday,Closer (Can Çelik),Abbedagge,FBI,huhi","Breathe,Wei (Yan Yang-Wei),Xiaohu,GALA,Ming (S...",0.0,4.0,0.0,1.0,3.0,8.0,42890.0,54884.0,4.0,17.0,2.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,2022 Season World Championship,Isurus,Saigon Buffalo,0,1,2022-09-30 22:29:00,29:12,29.200000,"Pantheon,Sylas,Seraphine,Tahm Kench,Taliyah","Sejuani,Graves,Azir,Amumu,Ornn","Zilean,Hecarim,LeBlanc,Kalista,Sett","Aatrox,Poppy,Viktor,Aphelios,Renata Glasc","ADD,Grell,seiya,Gavotto,Jelly (Son Ho-gyeong)","Hasmed,BeanJ,Froggy,Shogun (Nguyễn Văn Huy),Ta...",1.0,2.0,0.0,1.0,2.0,10.0,44891.0,61203.0,10.0,26.0,0.0,2.0,0.0,2.0
214,2022 Season World Championship,Saigon Buffalo,MAD Lions,0,1,2022-10-03 19:54:00,23:22,23.366667,"Sylas,Draven,Aatrox,Seraphine,Kalista","Aphelios,Caitlyn,Maokai,Camille,Nautilus","Lillia,Hecarim,Lissandra,Kai'Sa,Shen","Gnar,Bel'Veth,Vex,Varus,Tahm Kench","Hasmed,BeanJ,Froggy,Shogun (Nguyễn Văn Huy),Ta...","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser",1.0,1.0,0.0,1.0,0.0,9.0,36520.0,50497.0,9.0,19.0,0.0,2.0,0.0,1.0
215,2022 Season World Championship,MAD Lions,Saigon Buffalo,1,0,2022-10-03 19:54:00,23:22,23.366667,"Aphelios,Caitlyn,Maokai,Camille,Nautilus","Sylas,Draven,Aatrox,Seraphine,Kalista","Gnar,Bel'Veth,Vex,Varus,Tahm Kench","Lillia,Hecarim,Lissandra,Kai'Sa,Shen","Armut,Elyoya,Nisqy,UNF0RGIVEN,Kaiser","Hasmed,BeanJ,Froggy,Shogun (Nguyễn Văn Huy),Ta...",1.0,1.0,1.0,0.0,9.0,0.0,50497.0,36520.0,19.0,9.0,2.0,0.0,1.0,0.0
216,2022 Season World Championship,Saigon Buffalo,MAD Lions,0,1,2022-10-03 20:41:00,33:25,33.416667,"Draven,Sylas,Gnar,Tahm Kench,Braum","Aphelios,Caitlyn,Aatrox,Gwen,Renata Glasc","Fiora,Pantheo

In [13]:
games.sort_values(by='DateTime UTC').tail()

,OverviewPage,Team,Opponent,Win,Loss,DateTime UTC,Gamelength,Gamelength Number,TeamBans,OpponentBans,TeamPicks,OpponentPicks,TeamPlayers,OpponentPlayers,TeamDragons,OpponentDragons,TeamBarons,OpponentBarons,TeamTowers,OpponentTowers,TeamGold,OpponentGold,TeamKills,OpponentKills,TeamRiftHeralds,OpponentHeralds,TeamInhibitors,OpponentInhibitors
30,2022 Season World Championship,DWG KIA,Gen.G,1,0,2022-10-22 23:12:00,26:08,26.133333,"Sylas,Fiora,Yuumi,Renekton,Vi","Graves,Aatrox,Caitlyn,Lissandra,Camille","Sejuani,Viego,Swain,Aphelios,Lulu","Maokai,Poppy,Azir,Lucian,Nami","Nuguri,Canyon,ShowMaker,deokdam,Kellin","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...",4.0,0.0,1.0,0.0,7.0,1.0,50199.0,39719.0,15.0,2.0,2.0,0.0,1.0,0.0
71,2022 Season World Championship,DWG KIA,Gen.G,1,0,2022-10-23 00:01:00,24:30,24.500000,"Sylas,Aatrox,Yuumi,Thresh,Camille","Graves,Caitlyn,Kayn,Taliyah,Ornn","Renekton,Viego,LeBlanc,Aphelios,Renata Glasc","Gnar,Sejuani,Azir,Jinx,Lulu","Nuguri,Canyon,ShowMaker,deokdam,Kellin","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...",3.0,1.0,1.0,0.0,8.0,0.0,49795.0,37105.0,13.0,2.0,2.0,0.0,1.0,0.0
70,2022 Season World Championship,Gen.G,DWG KIA,0,1,2022-10-23 00:01:00,24:30,24.500000,"Graves,Caitlyn,Kayn,Taliyah,Ornn","Sylas,Aatrox,Yuumi,Thresh,Camille","Gnar,Sejuani,Azir,Jinx,Lulu","Renekton,Viego,LeBlanc,Aphelios,Renata Glasc","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...","Nuguri,Canyon,ShowMaker,deokdam,Kellin",1.0,3.0,0.0,1.0,0.0,8.0,37105.0,49795.0,2.0,13.0,0.0,2.0,0.0,1.0
72,2022 Season World Championship,Gen.G,DWG KIA,1,0,2022-10-23 00:49:00,44:51,44.850000,"Graves,Viego,Caitlyn,Nidalee,Karthus","Sylas,Aatrox,Yuumi,Yone,Vi","Gnar,Sejuani,Viktor,Lucian,Nami","Renekton,Kayn,Syndra,Aphelios,Lulu","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...","Nuguri,Canyon,ShowMaker,deokdam,Kellin",3.0,3.0,1.0,1.0,10.0,3.0,81645.0,72744.0,18.0,11.0,2.0,0.0,2.0,0.0
73,2022 Season World Championship,DWG KIA,Gen.G,0,1,2022-10-23 00:49:00,44:51,44.850000,"Sylas,Aatrox,Yuumi,Yone,Vi","Graves,Viego,Caitlyn,Nidalee,Karthus","Renekton,Kayn,Syndra,Aphelios,Lulu","Gnar,Sejuani,Viktor,Lucian,Nami","Nuguri,Canyon,ShowMaker,deokdam,Kellin","Doran (Choi Hyeon-joon),Peanut,Chovy,Ruler,Leh...",3.0,3.0,1.0,1.0,3.0,10.0,72744.0,81645.0,11.0,18.0,0.0,2.0,0.0,2.0


In [12]:
games.to_csv('./csv/2022_worlds_matches.csv', index=False)